In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "meta-llama/Llama-3.3-70B-Instruct"  # LLama LLM model I am using for the aggregator

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)  # converts raw text into tokens that the model can understand
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")  # transformer-based model specifically designed for casual language modeling

# Create a text-generation pipeline using the loaded model
aggregator_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=model.device if hasattr(model, "device") else 0
)

def aggregate_response(user_prompt: str, expert_outputs: list[str], max_length: int = 512, temperature: float = 1e-10) -> str:
    """
    Aggregates the responses from multiple expert models into a coherent answer using a Llama-based model.

    Args:
        user_prompt (str): The original user prompt.
        expert_outputs (list[str]): A list of responses from expert models.
        max_length (int): The maximum length of the generated output.
        temperature (float): Sampling temperature for generation.

    Returns:
        str: A coherent aggregated answer.
    """
    # Combine expert responses into a single context string.
    combined_expert_context = "\n\n".join(
        [f"Expert {i+1}:\n{output}" for i, output in enumerate(expert_outputs)]
    )

    # Construct the aggregator prompt.
    aggregator_prompt = f"""
You are an aggregator LLM designed to combine multiple expert responses into one coherent, concise answer.
Below is the original user query and responses from several experts. Synthesize the information to produce a single, clear aggregated answer.

User Query:
{user_prompt}

Expert Responses:
{combined_expert_context}

Provide a clear and concise aggregated answer:
"""
    # Generate the aggregated answer using the Llama-based aggregator model.
    generated_output = aggregator_pipeline(
        aggregator_prompt,
        max_length=max_length,
        temperature=temperature,
        do_sample=True  # Set to False for deterministic output
    )

    # The output is a list of dictionaries; extract the generated text.
    aggregated_answer = generated_output[0]['generated_text']

    # triming the prompt from the aggregated answer
    if aggregator_prompt.strip() in aggregated_answer:
        aggregated_answer = aggregated_answer.split(aggregator_prompt.strip())[-1].strip()

    return aggregated_answer

# Example usage:
if __name__ == "__main__":
    # Original user prompt
    user_query = "How can I optimize the performance of my deep learning models in production?"

    # Sample responses from expert models. The router should input these responses into the aggregator at a later step
    expert_responses = [
        "Consider using model quantization and pruning techniques to reduce model size and inference latency.",
        "Implementing efficient serving architectures such as TensorRT or ONNX Runtime can boost performance.",
        "Optimizing data pipelines and leveraging hardware accelerators like GPUs or TPUs will further improve throughput."
    ]

    # Get the aggregated answer using the Llama model.
    final_answer = aggregate_response(user_query, expert_responses)
    print("Aggregated Answer:\n", final_answer)
